##### Importamos las librerias Necesarias

In [2]:
import pandas as pd
import re
#import numpy as np
from datetime import datetime

##### Creamos el Dataset con el archivo movies_dataset.csv

In [3]:
df = pd.read_csv('Dataset/movies_dataset.csv')

/var/folders/mm/qj2l_7611k7cdqq50jvw32nc0000gn/T/ipykernel_2665/2393210234.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('Dataset/movies_dataset.csv')


In [4]:
df = df.drop(df[df['id'] == '1997-08-20'].index)
df = df.drop(df[df['id'] == '2012-09-29'].index)
df = df.drop(df[df['id'] == '2014-01-01'].index)

1. Algunos campos, como belongs_to_collection, production_companies y otros (ver diccionario de datos) están anidados, esto es o bien tienen un diccionario o una lista como valores en cada fila, ¡deberán desanidarlos para poder y unirlos al dataset de nuevo hacer alguna de las consultas de la API! O bien buscar la manera de acceder a esos datos sin desanidarlos.

In [5]:
# Definir el patrón de expresión regular
patron = r"'name': '(.*?)',"

# Función para extraer el nombre usando expresiones regulares
def extraer_nombre_coleccion(valor):
    if pd.isnull(valor):
        return ""
    resultado = re.search(patron, valor)
    if resultado:
        return resultado.group(1)
    else:
        return ""

# Aplicar la función a toda la columna 'belongs_to_collection' y asignar los resultados a la misma columna
df['belongs_to_collection'] = df['belongs_to_collection'].apply(extraer_nombre_coleccion)

# Imprimir el DataFrame resultante
df['belongs_to_collection'].head()

0              Toy Story Collection
1                                  
2         Grumpy Old Men Collection
3                                  
4    Father of the Bride Collection
Name: belongs_to_collection, dtype: object

In [6]:
patron = r"'name': '(.*?)'},"

def extraer_nombres_generos(valor):
    if pd.isnull(valor):
        return ""
    resultado = re.findall(patron, valor)
    if resultado:
        return resultado
    else:
        return ""

# Aplicar la función a toda la columna 'genres' y asignar los resultados a la misma columna
df['genres'] = df['genres'].apply(extraer_nombres_generos)

# Imprimir el DataFrame resultante
df['genres'].head()

0     [Animation, Comedy]
1    [Adventure, Fantasy]
2               [Romance]
3         [Comedy, Drama]
4                        
Name: genres, dtype: object

In [7]:
patron = r"'name': '(.*?)',"

def extraer_nombres_companies(valor):
    if pd.isnull(valor):
        return ""
    resultado = re.findall(patron, valor)
    if resultado:
        return resultado
    else:
        return ""

# Aplicar la función a toda la columna 'genres' y asignar los resultados a la misma columna
df['production_companies'] = df['production_companies'].apply(extraer_nombres_companies)
# Imprimir el DataFrame resultante
df['production_companies']

0                                [Pixar Animation Studios]
1        [TriStar Pictures, Teitler Film, Interscope Co...
2                           [Warner Bros., Lancaster Gate]
3                 [Twentieth Century Fox Film Corporation]
4             [Sandollar Productions, Touchstone Pictures]
                               ...                        
45461                                                     
45462                                        [Sine Olivia]
45463                            [American World Pictures]
45464                                          [Yermoliev]
45465                                                     
Name: production_companies, Length: 45463, dtype: object

In [8]:
patron = r"name': '(.*?)'}"

def extraer_nombres_countries(valor):
    if pd.isnull(valor):
        return ""
    resultado = re.findall(patron, valor)
    if resultado:
        return resultado
    else:
        return ""

# Aplicar la función a toda la columna 'genres' y asignar los resultados a la misma columna
df['production_countries'] = df['production_countries'].apply(extraer_nombres_countries)

# Imprimir el DataFrame resultante
df['production_countries']

0        [United States of America]
1        [United States of America]
2        [United States of America]
3        [United States of America]
4        [United States of America]
                    ...            
45461                        [Iran]
45462                 [Philippines]
45463    [United States of America]
45464                      [Russia]
45465              [United Kingdom]
Name: production_countries, Length: 45463, dtype: object

In [9]:
patron = r"name': '(.*?)'}"

def extraer_nombres_languajes(valor):
    if pd.isnull(valor):
        return ''
    resultado = re.findall(patron, valor)
    if resultado:
        return resultado
    else:
        return ''

# Aplicar la función a toda la columna 'genres' y asignar los resultados a la misma columna
df['spoken_languages'] = df['spoken_languages'].apply(extraer_nombres_languajes)

# Imprimir el DataFrame resultante
df['spoken_languages']

0                  [English]
1        [English, Français]
2                  [English]
3                  [English]
4                  [English]
                ...         
45461                [فارسی]
45462                     []
45463              [English]
45464                       
45465              [English]
Name: spoken_languages, Length: 45463, dtype: object

2. Los valores nulos de los campos revenue, budget deben ser rellenados por el número 0.

In [10]:
num_nulos_budget = df['budget'].isnull().sum()
num_nulos_revenue = df['revenue'].isnull().sum()
print("nulos budget:",num_nulos_budget,"nulos revenue:", num_nulos_revenue)

nulos budget: 0 nulos revenue: 3


In [11]:
df['budget'] = df['budget'].fillna(0)
df['revenue'] = df['revenue'].fillna(0)

In [12]:
num_nulos_budget = df['budget'].isnull().sum()
num_nulos_revenue = df['revenue'].isnull().sum()
print("nulos budget:",num_nulos_budget,"nulos revenue:", num_nulos_revenue)

nulos budget: 0 nulos revenue: 0


3. Los valores nulos del campo release date deben eliminarse.

In [13]:
num_nulos_release_date = df['release_date'].isnull().sum()
num_nulos_release_date

87

In [14]:
df= df.dropna(subset=['release_date'])

In [15]:
num_nulos_release_date = df['release_date'].isnull().sum()
num_nulos_release_date

0

4. De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

In [16]:
# Definir función para convertir la fecha en formato de cadena
def voltear_fecha(fecha_str):
    try:
        fecha_obj = datetime.strptime(fecha_str, "%d-%m-%Y")
        return fecha_obj.strftime("%Y-%m-%d")
    except ValueError:
        return fecha_str

# Aplicar la función a toda la columna 'Fecha'
df['release_date'] = df['release_date'].apply(voltear_fecha)

# Imprimir el DataFrame resultante
df['release_date']

0        1995-10-30
1        1995-12-15
2        1995-12-22
3        1995-12-22
4        1995-02-10
            ...    
45460    1991-05-13
45462    2011-11-17
45463    2003-08-01
45464    1917-10-21
45465    2017-06-09
Name: release_date, Length: 45376, dtype: object

5. Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

In [17]:
# Definir función para calcular el retorno
def calcular_retorno(row):
    if float(row['revenue']) == 0 or float(row['budget']) == 0:
        return 0
    else:
        return float(row['revenue']) / float(row['budget'])
    
df['return'] = df.apply(calcular_retorno, axis=1)

6. Eliminar las columnas que no serán utilizadas, video,imdb_id,adult,original_title,poster_path y homepage.

In [18]:
df = df.drop('video', axis=1)
df = df.drop('imdb_id', axis=1)
df = df.drop('adult', axis=1)
df = df.drop('original_title', axis=1)
df = df.drop('poster_path', axis=1)
df = df.drop('homepage', axis=1)

Agregamos la columna de los directores desde el archivo csv de credits.

In [19]:
df_credits = pd.read_csv('Dataset/credits.csv')
patron = r"'Director', 'name': '(.*?)', "

def extraer_nombres_director(valor):
    if pd.isnull(valor):
        return ""
    resultado = re.findall(patron, valor)
    if resultado:
        return resultado
    else:
        return ""

df['id'] = df['id'].astype(int)
df_merged = pd.merge(df, df_credits, on='id')
df_merged['director'] = df_merged['crew'].apply(extraer_nombres_director)

print(df_merged['director'])

0           [John Lasseter]
1            [Joe Johnston]
2           [Howard Deutch]
3         [Forest Whitaker]
4           [Charles Shyer]
                ...        
45446          [John Irvin]
45447            [Lav Diaz]
45448      [Mark L. Lester]
45449    [Yakov Protazanov]
45450       [Daisy Asquith]
Name: director, Length: 45451, dtype: object


In [20]:
print(df_merged.at[189, 'director'])

In [21]:
df_merged = df_merged.drop('cast', axis=1)
df_merged = df_merged.drop('crew', axis=1)

In [22]:
df_merged

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,return,director
0,Toy Story Collection,30000000,"[Animation, Comedy]",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,[Pixar Animation Studios],[United States of America],1995-10-30,373554033.0,81.0,[English],Released,NaN,Toy Story,7.7,5415.0,12.451801,[John Lasseter]
1,,65000000,"[Adventure, Fantasy]",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],1995-12-15,262797249.0,104.0,"[English, Français]",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,4.043035,[Joe Johnston]
2,Grumpy Old Men Collection,0,[Romance],15602,en,A family wedding reignites the ancient feud be...,11.7129,"[Warner Bros., Lancaster Gate]",[United States of America],1995-12-22,0.0,101.0,[English],Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,0.000000,[Howard Deutch]
3,,16000000,"[Comedy, Drama]",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[Twentieth Century Fox Film Corporation],[United States of America],1995-12-22,81452156.0,127.0,[English],Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,5.090760,[Forest Whitaker]
4,Father of the Bride Collection,0,,11862,en,Just when George Banks has recovered from his ...,8.387519,"[Sandollar Productions, Touchstone Pictures]",[United States of America],1995-02-10,76578911.0,106.0,[English],Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,0.000000,[Charles Shyer]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45446,,0,"[Drama, Action]",30840,en,"Yet another version of the classic epic, with ...",5.683753,"[Westdeutscher Rundfunk (WDR), Working Title F...","[Canada, Germany, United Kingdom, United State...",1991-05-13,0.0,104.0,[English],Released,NaN,Robin Hood,5.7,26.0,0.000000,[John Irvin]
45447,,0,,111109,tl,An artist struggles to finish his work while a...,0.178241,[Sine Olivia],[Philippines],2011-11-17,0.0,360.0,[],Released,NaN,Century of Birthing,9.0,3.0,0.000000,[Lav Diaz]
45448,,0,"[Action, Drama]",67758,en,"When one of her hits goes wrong, a professiona...",0.903007,[American World Pictures],[United States of America],2003-08-01,0.0,90.0,[English],Released,A deadly game of wits.,Betrayal,3.8,6.0,0.000000,[Mark L. Lester]
45449,,0,,227506,en,"In a small town live two brothers, one a minis...",0.003503,[Yermoliev],[Russia],1917-10-21,0.0,87.0,,Released,NaN,Satan Triumphant,0.0,0.0,0.000000,[Yakov Protazanov]


In [23]:
df_merged['original_language'] = df['original_language'].fillna('')
df_merged['overview'] = df['overview'].fillna('')
df_merged['production_countries'] = df['production_countries'].fillna('vacio')
df_merged['original_language'] = df_merged['original_language'].fillna('')

In [24]:
df_merged.to_csv('Dataset/df_limpio.csv', index=False)